In [42]:
import sys
import os
import pandas as pd
sys.path.append("scripts")

In [43]:

from combineSingleDB import load_and_aggregate_sentiment, merge_with_price
# Paths
sent_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/news/TSLA_sentiment_combined.csv"
price_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features.csv"  # Must be minute-level
output_path = "/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv"

# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(sent_path, resample_interval="1min")
sentiment_df.tail()


,Datetime,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement
10010,2025-06-18 16:59:00+00:00,0.000524,0.994288,0.005187,0.359341,0.558448,0.082211,0.010292,0.0,0.989708,0.123386,0.204402
10011,2025-06-18 17:00:00+00:00,0.000524,0.994288,0.005187,0.359341,0.558448,0.082211,0.010292,0.0,0.989708,0.123386,0.204402
10012,2025-06-18 17:01:00+00:00,0.000524,0.994288,0.005187,0.359341,0.558448,0.082211,0.010292,0.0,0.989708,0.123386,0.204402
10013,2025-06-18 17:02:00+00:00,0.000524,0.994288,0.005187,0.359341,0.558448,0.082211,0.010292,0.0,0.989708,0.123386,0.204402
10014,2025-06-18 17:03:00+00:00,0.000524,0.994288,0.005187,0.359341,0.558448,0.082211,0.010292,0.0,0.989708,0.123386,0.204402


In [44]:

from combineSingleDB import merge_with_price_incremental

# Get last timestamp in existing merged file
if os.path.exists(output_path):
    last_ts = pd.read_csv(output_path)["Datetime"].max()
else:
    last_ts = None

# Run the updated merge
merge_with_price_incremental(sentiment_df, price_path, output_path, last_timestamp=last_ts)

# View output
df = pd.read_csv(output_path)
df.tail()

✅ Updated dataset written to /Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features_full.csv with 0 new rows.


/Users/natwat/Desktop/CPSC_Projects/Trader/scripts/combineSingleDB.py:114: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label
1146,2025-06-18 19:55:00+00:00,322.179993,323.059998,322.089996,322.769989,326874.0,323.249747,0.010292,0.0,0.989708,0.359341,0.558448,0.082211,0.000524,0.994288,0.005187,0.123386,0.204402,0.000000,0
1147,2025-06-18 19:56:00+00:00,322.179993,322.359985,322.000000,322.180115,255342.0,323.147866,0.010292,0.0,0.989708,0.359341,0.558448,0.082211,0.000524,0.994288,0.005187,0.123386,0.204402,-0.000427,0
1148,2025-06-18 19:57:00+00:00,322.042297,322.281891,321.920013,322.200012,199259.0,323.042574,0.010292,0.0,0.989708,0.359341,0.558448,0.082211,0.000524,0.994288,0.005187,0.123386,0.204402,0.000239,1
1149,2025-06-18 19:58:00+00:00,322.119293,322.170013,321.799988,322.041504,282674.0,322.954642,0.010292,0.0,0.989708,0.359341,0.558448,0.082211,0.000524,0.994288,0.005187,0.123386,0.204402,-0.000075,0
1150,2025-06-18 19:59:00+00:00,322.095001,322.200012,321.750000,322.059998,705692.0,322.872772,0.010292,0.0,0.989708,0.359341,0.558448,0.082211,0.000524,0.994288,0.005187,0.123386,0.204402,NaN,0


In [45]:
import ta

df = pd.read_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/features/TSLA_features.csv", parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  



In [46]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]

In [47]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [48]:
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/Trader/data/final/TSLA_ready.csv", index=False)


In [49]:
df.tail()

,Datetime,Close,High,Low,Open,Volume,EMA20,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
